In [1]:
#Based on this tutorial - http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html

import pandas as pd
import pylab as pl
import numpy as np 
import geopandas as gp
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Data munging for all DV routes

In [2]:
demographics = pd.read_excel('../data/DemoData.xlsx')
demographics['tract'] = demographics.ID.astype(str).str.zfill(9)

In [3]:
shp = gp.GeoDataFrame.from_file('../data/merged.json')

shp = shp.merge(demographics, on='tract')

shp.rename(columns={
    'Destination?': 'is_dest',
    'Origin?': 'is_orig',
    'Total Population': 'pop',
    '18 to 24 yrs': '_18_to_24',
    '65 and Above': 'above_65',
    'Median Age': 'age',
    'Non-white': 'nonwhite',
    'English less than "very well"': 'nonenglish',
    'Annaul Individual Income below 10000': 'income_below_10000',
    'Annaul Individual Income below 15000': 'income_below_15000',
    'Median Income': 'income',
    'Below 100 percent of the poverty level': 'below_pov',
    '100 to 149 percent of the poverty level': '_100_149_pov',
    'At or above 150 percent of the poverty level': 'above_150_pov',
    'Citizenship status': 'noncitizen',
    'Place of Birth': 'foreignborn'
}, inplace=True)

In [4]:
lehd = pd.read_csv('../data/merged.csv', dtype={'tract': str})

lehd = lehd.drop([u'Unnamed: 0', u'Both', u'Destination?', u'DollarVanLine', u'Origin?',
       u'average_commute_time', u'average_walk_distance', u'geometry',
       u'number_of_commuters',  u'w_county_tract', u'h_county_tract'], 1)

shp = shp.merge(lehd, on='tract')

In [5]:
shp.dropna(how="all", inplace=True)

In [6]:
data = shp.ix[:,4:]
labels = shp['DollarVanLine']

In [7]:
data = data[[
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                  u'noncitizen',           u'foreignborn',
              u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]]

#Dropping poverty because nans while computing covariance matrix
data_std = data

In [8]:
# Standardize Value Range

for c in [
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                 u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]:
    data_std[c] = ((data_std[c] - data_std[c].mean())/data_std[c].std())
    


count    2166.000000
mean        0.085411
std         0.279557
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Count_, dtype: float64

### Data munging for individual routes

In [9]:
import numpy as np
mean_vec = np.mean(data_std, axis=0)
cov_mat = (data_std - mean_vec).T.dot((data_std - mean_vec)) / (data_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)

Covariance matrix 
                       average_commute_time  average_walk_distance  \
average_commute_time               1.000000               0.909334   
average_walk_distance              0.909334               1.000000   
number_of_commuters               -0.630156              -0.577657   
pop                               -0.551228              -0.511883   
_18_to_24                         -0.038245              -0.041112   
above_65                           0.131807               0.088601   
age                                0.184564               0.146259   
nonwhite                           0.238677               0.178632   
nonenglish                         0.045640               0.039278   
income_below_10000                 0.059571               0.056164   
income_below_15000                 0.070413               0.065146   
income                            -0.275443              -0.218678   
noncitizen                         0.123905               0.097909   
f

In [10]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
cor_mat1 = np.corrcoef(data_std.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat1)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[ -2.47445519e-01  -7.92389923e-02   3.98891360e-02   1.44456712e-01
   -5.59523726e-02  -3.14595629e-01  -3.56736440e-01  -3.25650939e-01
    7.32392553e-02   7.21167834e-03  -2.51928454e-02  -8.27282855e-03
    6.35640296e-02  -1.14438618e-01  -2.55522444e-02   3.01768359e-02
   -8.72312883e-03   6.65510038e-02  -6.56776545e-01   2.21491949e-01
   -2.30323662e-01   6.52825242e-02  -8.70217503e-02   6.56378081e-04
    5.35663439e-16   1.46146197e-15   6.01704256e-16  -3.48498462e-16]
 [ -2.27509894e-01  -8.13991127e-02   3.97705643e-02   1.23014081e-01
   -7.07372860e-02  -3.15351578e-01  -3.77590071e-01  -4.45151718e-01
    1.83433899e-01   6.70096781e-02  -4.90871193e-02   8.28539377e-02
    2.50061862e-02   1.63787943e-01  -2.71127482e-03   8.60145288e-02
    3.67168627e-03  -5.38408032e-02   5.54326860e-01  -2.20381403e-01
    1.89851158e-01  -4.82528100e-02   1.76925004e-02   9.73762318e-04
   -1.76245771e-16  -3.56380748e-16  -5.46098353e-16   6.57566886e-16]
 [  

In [11]:
cor_mat2 = np.corrcoef(data.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat2)


In [12]:
u,s,v = np.linalg.svd(data_std.T)

In [45]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0], i[1])# max(i[1]), np.mean(i[1]), np.std(i[1]), min(i[1]) )

Eigenvalues in descending order:
(9.0535763757323569, array([-0.24744552, -0.22750989,  0.26752878,  0.23646118, -0.01404069,
       -0.00187555, -0.002666  , -0.08972109, -0.05356957, -0.06964585,
       -0.07788279,  0.15675925, -0.05945666, -0.01701496,  0.18997601,
        0.19389964,  0.18683487,  0.18248547,  0.19743281,  0.19256351,
        0.17890414,  0.28774482,  0.27153521,  0.28622749,  0.25451958,
        0.2163573 ,  0.1689631 ,  0.28377035]))
(6.4318487880283914, array([-0.07923899, -0.08139911,  0.0892596 ,  0.23038217,  0.00260542,
       -0.03469308, -0.05685277,  0.07129696,  0.07243225,  0.0961228 ,
        0.11140478, -0.14363678, -0.02807025, -0.04671527, -0.30556717,
       -0.29327885, -0.30498035, -0.29766506, -0.26442452, -0.28911836,
       -0.30465761,  0.18592327,  0.17830476,  0.17928188,  0.17638796,
        0.24007335,  0.25246872,  0.08780496]))
(3.6984014753131573, array([ 0.03988914,  0.03977056,  0.06691235, -0.06999215, -0.29829137,
        0.221686

In [20]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(28,1),
                      eig_pairs[1][1].reshape(28,1)))

print('Matrix W:\n', matrix_w)

('Matrix W:\n', array([[-0.24744552, -0.07923899],
       [-0.22750989, -0.08139911],
       [ 0.26752878,  0.0892596 ],
       [ 0.23646118,  0.23038217],
       [-0.01404069,  0.00260542],
       [-0.00187555, -0.03469308],
       [-0.002666  , -0.05685277],
       [-0.08972109,  0.07129696],
       [-0.05356957,  0.07243225],
       [-0.06964585,  0.0961228 ],
       [-0.07788279,  0.11140478],
       [ 0.15675925, -0.14363678],
       [-0.05945666, -0.02807025],
       [-0.01701496, -0.04671527],
       [ 0.18997601, -0.30556717],
       [ 0.19389964, -0.29327885],
       [ 0.18683487, -0.30498035],
       [ 0.18248547, -0.29766506],
       [ 0.19743281, -0.26442452],
       [ 0.19256351, -0.28911836],
       [ 0.17890414, -0.30465761],
       [ 0.28774482,  0.18592327],
       [ 0.27153521,  0.17830476],
       [ 0.28622749,  0.17928188],
       [ 0.25451958,  0.17638796],
       [ 0.2163573 ,  0.24007335],
       [ 0.1689631 ,  0.25246872],
       [ 0.28377035,  0.08780496]]))


In [27]:
Y = data_std.dot(matrix_w)

In [28]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(data_std)

In [ ]:
Y_sklearn

In [38]:
from sklearn.decomposition import SparsePCA as sparsePCA
sparse_pca = sparsePCA(n_components=4)
sparse_pca.fit(data_std)
Y_sparse = sparse_pca.transform(data_std)

In [39]:
Y_sparse[0:4]

array([[-0.0070289 ,  0.00197309, -0.03405135, -0.01227559],
       [ 0.03357797,  0.00271299, -0.02595517, -0.02577511],
       [ 0.05683921,  0.00040607, -0.05654606, -0.03619672],
       [ 0.06356183, -0.16317894,  0.05018263,  0.02857119]])

In [63]:
routes = pd.read_csv('../data/csv/masterroutes.csv', dtype={'CT2010': str, 'BoroCode': str})

routes['county_code'] = ""
routes['county_code'][routes['BoroCode'] == '1'] = '061' # Manhattan
routes['county_code'][routes['BoroCode'] == '2'] = '005' # Bronx
routes['county_code'][routes['BoroCode'] == '3'] = '047' # Brooklyn
routes['county_code'][routes['BoroCode'] == '4'] = '081' # Queens
routes['county_code'][routes['BoroCode'] == '5'] = '085' # Staten Island

routes['tract'] = routes['county_code'] + routes['CT2010']


routes.head()

/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveat

,FID,FID_1,CTLabel,BoroCode,BoroName,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,...,Sum_Id,Destination,FlatbushDest,ChinaDest,Jamica,OriginChina,OriginFlat,OriginJamica,county_code,tract
0,0,0,9.00,5,Staten Island,000900,5000900,I,SI22,West New Brighton-New Brighton-St. George,...,0,0,0,0,0,0,0,0,085,085000900
1,1,1,125.00,5,Staten Island,012500,5012500,I,SI22,West New Brighton-New Brighton-St. George,...,0,0,0,0,0,0,0,0,085,085012500
2,2,2,133.01,5,Staten Island,013301,5013301,E,SI22,West New Brighton-New Brighton-St. George,...,0,0,0,0,0,0,0,0,085,085013301
3,3,3,197.00,5,Staten Island,019700,5019700,I,SI07,Westerleigh,...,0,0,0,0,0,0,0,0,085,085019700
4,4,4,64.00,5,Staten Island,006400,5006400,I,SI14,Grasmere-Arrochar-Ft. Wadsworth,...,0,0,0,0,0,0,0,0,085,085006400


In [53]:
shp['tract']

0       061000201
1       061000202
2       061000600
3       061000700
4       061000800
5       061000900
6       061001001
7       061001002
8       061001200
9       061001300
10      061001401
11      061001402
12      061001501
13      061001502
14      061001600
15      061001800
16      061002000
17      061002100
18      061002201
19      061002202
20      061002400
21      061002500
22      061002601
23      061002602
24      061002700
25      061002800
26      061002900
27      061003001
28      061003002
29      061003100
          ...    
2087    085020100
2088    085020700
2089    085020801
2090    085020803
2091    085020804
2092    085021300
2093    085022300
2094    085022600
2095    085022800
2096    085023100
2097    085023900
2098    085024401
2099    085024402
2100    085024700
2101    085024800
2102    085025100
2103    085027301
2104    085027302
2105    085027702
2106    085027704
2107    085027705
2108    085027706
2109    085027900
2110    085029102
2111    08

In [56]:
shp.columns

Index([                 u'Both',               u'is_dest',
               u'DollarVanLine',               u'is_orig',
        u'average_commute_time', u'average_walk_distance',
                    u'geometry',   u'number_of_commuters',
                       u'tract',                    u'ID',
                   u'Geography',                   u'pop',
                   u'_18_to_24',              u'above_65',
                         u'age',              u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                   u'below_pov',          u'_100_149_pov',
               u'above_150_pov',            u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29